In [2]:
import numpy as np

from rdkit.Chem import AllChem as chem
from rdkit.Chem import rdmolops as ops

from torch_geometric.data import Data

import torch
import torch.nn.functional as F

# Calculate atom features for GNN input
def get_atom_features(atom : chem.Atom):
    atom_num = F.one_hot(
        torch.tensor(atom.GetAtomicNum()), num_classes=60)
    valence = F.one_hot(
        torch.tensor(atom.GetTotalValence()), num_classes=8)
    aromaticity = torch.tensor([int(atom.GetIsAromatic())])
    mol_weight = torch.tensor([atom.GetMass()])
    
    return torch.cat((atom_num, valence, aromaticity, mol_weight))

# Calculate bond features for GNN input
def get_bond_features(bond : chem.Bond):
    match bond.GetBondType():
        case chem.BondType.SINGLE:
            bond_type = 0
        case chem.BondType.DOUBLE:
            bond_type = 1
        case chem.BondType.TRIPLE:
            bond_type = 2
        case _:
            bond_type = 3
    bond_type = F.one_hot(torch.tensor(bond_type), num_classes=4)
    aromaticity = torch.tensor([int(bond.GetIsAromatic())])
    
    return torch.cat((bond_type, aromaticity))
    

In [3]:
# Create data set of featurized molecular graphs from SMILES input 
def create_data_set(smiles_list, labels):
    data_list = []
    
    for (smiles, label) in zip(smiles_list, labels):
        mol = chem.MolFromSmiles(smiles)
        n_nodes = mol.GetNumAtoms()
        n_edges = 2*mol.GetNumBonds()
        features_test = "O=O"
        features_test_mol = chem.MolFromSmiles(features_test)
        n_node_features = len(
            get_atom_features(features_test_mol.GetAtomWithIdx(0)))
        n_bond_features = len(
            get_bond_features(features_test_mol.GetBondWithIdx(0)))
        
        x = torch.tensor(
            np.zeros((n_nodes, n_node_features)), dtype = torch.float)
        
        for atom in mol.GetAtoms():
            x[atom.GetIdx(), :] = get_atom_features(atom)
            
        (rows, cols) = np.nonzero(ops.GetAdjacencyMatrix(mol))
        torch_rows = torch.tensor(rows.astype(np.int64)).to(torch.long)
        torch_cols = torch.tensor(cols.astype(np.int64)).to(torch.long)
        e = torch.stack((torch_rows, torch_cols))
        
        ef = torch.tensor(
            np.zeros((n_edges, n_bond_features)), dtype = torch.float)
        
        for (k, (i, j)) in enumerate(zip(rows, cols)):
            ef[k] = get_bond_features(
                mol.GetBondBetweenAtoms(int(i), int(j)))
        
        label_tensor = torch.tensor(np.array([label]), dtype = torch.float)
        
        data_list.append(
            Data(x = x, edge_index = e, edge_attr = ef, y = label_tensor))
        
    return data_list

In [7]:
import pandas as pd

# Load dataset
lipo = pd.read_csv('Lipophilicity.csv', index_col=0)
data_set = create_data_set(lipo['smiles'], lipo['exp'])

In [116]:
from model import GCN

# Create model from model.py
net = GCN(
    in_channels=-1, hidden_channels=50, 
    num_layers = 1, jk='last', dropout=0.5)
net

GCN(-1, 50, num_layers=1)

In [117]:
from torch.nn import MSELoss
from tqdm import tqdm

# Establish loss function and training optimizer
loss = MSELoss()
optimizer = torch.optim.Adam(params=net.parameters(), lr=0.001)

# Set model to training mode
net.train()

# Run training loop
for epoch in range(5):
    train_loss = []
    
    for data in tqdm(data_set):
        # Predict from input
        preds = net(data.x, data.edge_index)
        
        # Calculate loss between prediction and target output
        l = loss(preds, data.y.reshape(()))
        
        # Save training loss
        train_loss += [float(l)]
        
        # Calculate gradients of model parameters and adjust 
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    
    # Print average training loss for each epoch
    print(f"Epoch: {epoch}, \
              Training loss: {np.mean(train_loss)}")

100%|██████████| 4200/4200 [00:11<00:00, 354.07it/s]


Epoch: 0,               Training loss: 12750.503911493233


100%|██████████| 4200/4200 [00:17<00:00, 234.59it/s]


Epoch: 1,               Training loss: 4.73064059163662


100%|██████████| 4200/4200 [00:17<00:00, 238.28it/s]


Epoch: 2,               Training loss: 3.1521114207325827


100%|██████████| 4200/4200 [00:11<00:00, 353.99it/s]


Epoch: 3,               Training loss: 1.9232074729229238


100%|██████████| 4200/4200 [00:12<00:00, 326.80it/s]

Epoch: 4,               Training loss: 1.770746563666239


In [125]:
# Print predicted and actual lipophilicity for 20 molecules from training dataset
net.eval()
for i in range(0, 20):
    print('Predicted lipophilicity:',
        f'{net(data_set[i].x, data_set[i].edge_index)},',
        f'Actual: {data_set[i].y[0]}')


Predicted lipophilicity: 2.746176242828369, Actual: 3.5399999618530273
Predicted lipophilicity: 2.221467971801758, Actual: -1.1799999475479126
Predicted lipophilicity: 2.0041699409484863, Actual: 3.690000057220459
Predicted lipophilicity: 2.3940212726593018, Actual: 3.369999885559082
Predicted lipophilicity: 2.70646595954895, Actual: 3.0999999046325684
Predicted lipophilicity: 2.602780342102051, Actual: 3.140000104904175
Predicted lipophilicity: 2.3352832794189453, Actual: -0.7200000286102295
Predicted lipophilicity: 2.8879833221435547, Actual: 0.3400000035762787
Predicted lipophilicity: 1.7532542943954468, Actual: 3.049999952316284
Predicted lipophilicity: 2.097764015197754, Actual: 2.25
Predicted lipophilicity: 2.2723095417022705, Actual: 1.5099999904632568
Predicted lipophilicity: 3.271427631378174, Actual: 2.609999895095825
Predicted lipophilicity: 0.5100199580192566, Actual: -0.07999999821186066
Predicted lipophilicity: 2.611186981201172, Actual: 1.9500000476837158
Predicted lipop

In [126]:
# Predict lipophilicty for unseen molecule
morphine = create_data_set(
    ['CN1CCC23C4C1CC5=C2C(=C(C=C5)O)OC3C(C=C4)O'], [0.87])[0]
print('Predicted lipophilicity:',
    f'{net(morphine.x, morphine.edge_index)},',
    f'Actual: {morphine.y[0]}')

Predicted lipophilicity: 1.552609920501709, Actual: 0.8700000047683716
